In [1]:
# environment: sceneGraphs_Gaia

# TODO: how to compute the distance between objects: centroid, distance among single points, ...

In [2]:
import numpy as np
import os
import open3d as o3d

import scannet200_constants

In [3]:
pcd_mask3D = o3d.io.read_point_cloud(os.path.join("/local/home/gmarsich/data2TB/LabelMaker/processed_ARKitScenes/40753679/intermediate/scannet200_mask3d_1/mesh_labelled.ply")) # TODO TOSET: change the name of the point cloud to open
o3d.visualization.draw_geometries([pcd_mask3D])

In [4]:
base_path = "/local/home/gmarsich/data2TB/LabelMaker/processed_ARKitScenes/40753679/intermediate/scannet200_mask3d_1" # TODO TOSET
path_predictions = os.path.join(base_path, "predictions.txt") # TODO TOSET: change if necessary

In [5]:
path_pred_masks = "/local/home/gmarsich/data2TB/LabelMaker/processed_ARKitScenes/40753679/intermediate/scannet200_mask3d_1/pred_mask" #TODO TOSET
all_files = os.listdir(path_pred_masks)
txt_files = [f for f in all_files if f.endswith('.txt')]

def extract_number(filename):
    return int(os.path.splitext(filename)[0])

txt_files.sort(key=extract_number)

sorted_txt_paths = [os.path.join(path_pred_masks, f) for f in txt_files]

In [6]:
#
# Get the list of objects that appear in the segmentation (use the files provided from the work of Mask3D)
#

def build_legend(path_predictions):
    with open(path_predictions, 'r') as file:
        lines = file.readlines()

    predictions = []

    for line in lines:
        parts = line.strip().split()
        filename = parts[0]
        file_number = filename.split('/')[1].split('.')[0]
        object_ID = int(parts[1])
        confidence = float(parts[2])

        predictions.append([file_number, object_ID, confidence])

    # Build a list with the info that I need
    objects = []

    for prediction in predictions:
        object_ID = prediction[1]
        
        # Find the object_ID in the objects list
        found = False
        for obj in objects:
            if obj[0] == object_ID:
                obj[1] += 1
                found = True
                break
        
        # If the object_ID was not found, add it to the list with a count of 1
        if not found:
            objects.append([object_ID, 1])


    #
    # Build a big table with the correspondences between VALID_CLASS_IDS_200, CLASS_LABELS_200 and SCANNET_COLOR_MAP_200 from scannet200_constants
    #

    table_scannet200 = []

    for class_id, label in zip(scannet200_constants.VALID_CLASS_IDS_200, scannet200_constants.CLASS_LABELS_200):
        color = scannet200_constants.SCANNET_COLOR_MAP_200[class_id]
        table_scannet200.append((class_id, label, color))

    # An alternative could be to get the colours from the point cloud and search for their assciated IDs (and name of the object) on
        # https://github.com/ScanNet/ScanNet/blob/master/BenchmarkScripts/ScanNet200/scannet200_constants.py


    #
    # Use the big table to add information to the list objects
    #

    # Add label and colour
    for obj in objects:
        object_ID = obj[0]
        
        for entry in table_scannet200:
            class_id, label, color = entry
            if object_ID == class_id:
                obj.append(label)
                obj.append(color)
                break

    # Sort the objects list by the ID (first element of each sublist)
    objects.sort(key=lambda x: x[0])
    return objects

objects = build_legend(path_predictions)

In [7]:
with open(path_predictions, 'r') as file:
    lines = file.readlines()

list_labels = [] # will contain a list of [label, position, color]

def get_centroid(path_pred_mask, pcd_mask3D):
    points = np.asarray(pcd_mask3D.points)
    with open(path_pred_mask, 'r') as f:
        mask = np.array([int(line.strip()) for line in f])
    assert len(mask) == len(points)
    selected_points = points[mask == 1]
    centroid = np.mean(selected_points, axis=0)
    return centroid


for i, path_pred_mask in enumerate(sorted_txt_paths):
    parts = lines[i].strip().split()
    object_ID = int(parts[1])
    label = None
    color = None
    position = get_centroid(path_pred_mask, pcd_mask3D)
    for index in range(len(objects)):
        if objects[index][0] == object_ID:
            label = objects[index][2]
            color = objects[index][3]
            #break
    list_labels.append([position, label, object_ID, color])

transposed_list_labels = [list(row) for row in zip(*list_labels)]

In [8]:
matrix_distances = np.full((len(list_labels), len(list_labels)), np.inf)

for i in range(len(matrix_distances)):
    for j in range(i + 1, len(matrix_distances[0])):
        matrix_distances[i][j] = np.linalg.norm(list_labels[i][0] - list_labels[j][0])


In [21]:
threshold = 2 # distance threshold in meters

# Function to create the graph with colors
def create_graph_with_colors(list_labels, matrix_distances):
    vertices = [label[0] for label in list_labels]
    colors = [np.array(label[3]) / 255.0 for label in list_labels]
    
    # Create a point cloud for the vertices
    points = o3d.geometry.PointCloud()
    points.points = o3d.utility.Vector3dVector(vertices)
    points.colors = o3d.utility.Vector3dVector(colors)

    # Create lines for the edges
    edges = []
    for i in range(len(matrix_distances)):
        for j in range(i + 1, len(matrix_distances[0])):
            if matrix_distances[i][j] <= threshold:
                edges.append([i, j])
    
    lines = o3d.geometry.LineSet()
    lines.points = o3d.utility.Vector3dVector(vertices)
    lines.lines = o3d.utility.Vector2iVector(edges)
    lines.paint_uniform_color([0, 0, 0])

    return points, lines


# Function to visualize with custom point size
def visualize_with_custom_point_size(points, lines, point_size=5):
    # Create a visualization window and set point size
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(points)
    vis.add_geometry(lines)

    # Get the render options and set point size
    render_option = vis.get_render_option()
    render_option.point_size = point_size  # Adjust the point size as needed

    # Run the visualization
    vis.run()
    vis.destroy_window()

# Create the graph
points, lines = create_graph_with_colors(list_labels, matrix_distances)

# Visualize the graph with custom point size
visualize_with_custom_point_size(points, lines, point_size=10)


Get the scene graph with the labels:

In [28]:
import pyviz3d.visualizer as viz
import numpy as np

# Assuming you have imported other necessary libraries and defined your functions as per your setup...

# First, we set up a visualizer
v = viz.Visualizer()

name = 'Point cloud'
point_positions = np.asarray(pcd_mask3D.points)
point_colors = (np.asarray(pcd_mask3D.colors) * 255).astype(np.uint8)
point_size = 10

# Here we add point clouds to the visualizer
v.add_points(name, point_positions, point_colors, point_size=point_size, visible=False)

v.add_labels(name ='Labels',
                 labels = transposed_list_labels[1],
                 positions = transposed_list_labels[0],
                 colors = transposed_list_labels[3],
                 visible=True)

# Assuming you have defined transposed_list_labels and matrix_distances

def create_graph_with_colors(list_labels, matrix_distances, threshold):
    vertices = [label[0] for label in list_labels]
    colors = [np.array(label[3]) / 255.0 for label in list_labels]

    # Create lines for the edges
    edges = []
    for i in range(len(matrix_distances)):
        for j in range(i + 1, len(matrix_distances[0])):
            if matrix_distances[i][j] <= threshold:
                edges.append([i, j])

    lines_start = np.array([vertices[i] for (i, j) in edges])
    lines_end = np.array([vertices[j] for (i, j) in edges])
    lines_colors = np.array([colors[i] for (i, j) in edges])

    return lines_start, lines_end, lines_colors

threshold = 2  
lines_start, lines_end, lines_colors = create_graph_with_colors(list_labels, matrix_distances, threshold)

v.add_lines(name='Graph Lines', lines_start=lines_start, lines_end=lines_end, colors=lines_colors, visible=True)

# When we added everything we need to the visualizer, we save it.
v.save('example_point_clouds')



************************************************************************
1) Start local server:
    cd /local/home/gmarsich/Desktop/Thesis/0Code_playground/SceneGraphs/example_point_clouds; python -m http.server 6008
2) Open in browser:
    http://localhost:6008
************************************************************************


In [38]:
import pyviz3d.visualizer as viz
import numpy as np

# Assuming you have imported other necessary libraries and defined your functions as per your setup...

# First, we set up a visualizer
v = viz.Visualizer()

name = 'Point cloud'
point_positions = np.asarray(pcd_mask3D.points)
point_colors = (np.asarray(pcd_mask3D.colors) * 255).astype(np.uint8)
point_size = 10

# Here we add point clouds to the visualizer
v.add_points(name, point_positions, point_colors, point_size=point_size, visible=False)

v.add_labels(name ='Labels',
                 labels = transposed_list_labels[1],
                 positions = transposed_list_labels[0],
                 colors = transposed_list_labels[3],
                 visible=True)

# Assuming you have defined transposed_list_labels and matrix_distances

def create_graph_with_colors(list_labels, matrix_distances, threshold):
    vertices = [label[0] for label in list_labels]
    colors = [np.array(label[3]) / 255.0 for label in list_labels]

    # Create lines for the edges
    edges = []
    for i in range(len(matrix_distances)):
        for j in range(i + 1, len(matrix_distances[0])):
            if matrix_distances[i][j] <= threshold:
                edges.append([i, j])

    lines_start = np.array([vertices[i] for (i, j) in edges])
    lines_end = np.array([vertices[j] for (i, j) in edges])
    lines_colors = np.ones(lines_start.shape, dtype=np.uint8) * np.array([255, 0, 0]) # red lines
    lines_colors = lines_colors.astype(np.uint8)

    return lines_start, lines_end, lines_colors

threshold = 2  
lines_start, lines_end, lines_colors = create_graph_with_colors(list_labels, matrix_distances, threshold)

v.add_lines(name='Graph Lines', lines_start=lines_start, lines_end=lines_end, colors=lines_colors, visible=True)

# When we added everything we need to the visualizer, we save it.
v.save('example_point_clouds')


[[255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]
 [255   0   0]]

************************************************************************
1) Start local server:
  